In [1]:
''' heavily borrowed from https://github.com/GalvanizeDataScience/lectures/blob/Denver/text-classification/frank-burkholder/naive_bayes_sklearn.ipynb'''



' heavily borrowed from https://github.com/GalvanizeDataScience/lectures/blob/Denver/text-classification/frank-burkholder/naive_bayes_sklearn.ipynb'

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [3]:
categories = [0, 1]

In [4]:
path_csv = 'data/tweets.csv'
df1 = pd.read_csv(path_csv)

#impute empty records
df = df1.fillna("None")


df1 = df[["sentiment", "description"]].copy()
# drop rows with "neutral" sentiment
df_content = df1[~df1.sentiment.str.contains("neutral")]
# print(df_content.head())

X = df_content["description"] #data
X.to_numpy()
print(X)

y= df_content["sentiment"].replace("positive", 1).replace("negative", 0)
y.to_numpy()
                                 

1        @VirginAmerica plus you've added commercials t...
3        @VirginAmerica it's really aggressive to blast...
4        @VirginAmerica and it's a really big bad thing...
5        @VirginAmerica seriously would pay $30 a fligh...
6        @VirginAmerica yes, nearly every time I fly VX...
                               ...                        
14633    @AmericanAir my flight was Cancelled Flightled...
14634           @AmericanAir right on cue with the delays👌
14635    @AmericanAir thank you we got on a different f...
14636    @AmericanAir leaving over 20 minutes Late Flig...
14638    @AmericanAir you have my money, you change my ...
Name: description, Length: 11541, dtype: object


array([1, 0, 0, ..., 1, 0, 0])

In [5]:
print(len(X)) #11541
print(len(y)) #11541

11541
11541


In [6]:
import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
new_stopwords = ['a']
new_stopwords_list = stop_words.union(new_stopwords)

count_vect = CountVectorizer(lowercase=True, tokenizer=None, stop_words = new_stopwords_list,  analyzer='word', max_df=1.0, min_df=1,  max_features=None) 
                             

count_vect.fit(X)

target_names = y

In [7]:
print(count_vect)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [8]:

X_counts = count_vect.transform(X)
print("The type of X_counts is {0}.".format(type(X_counts)))
print("The X matrix has {0} rows (documents) and {1} columns (words).".format(
        X_counts.shape[0], X_counts.shape[1]))

The type of X_counts is <class 'scipy.sparse.csr.csr_matrix'>.
The X matrix has 11541 rows (documents) and 12503 columns (words).


In [9]:
tfidf_transformer = TfidfTransformer(use_idf=True)
tfidf_transformer.fit(X_counts)
X_tfidf = tfidf_transformer.transform(X_counts)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y.values, test_size=0.2, random_state=42)

In [11]:
print(X_train.shape)
print(y_train.shape)

(9232, 12503)
(9232,)


In [12]:
print(X_test.shape)
print(y_test.shape)

(2309, 12503)
(2309,)


In [13]:
nb_model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nb_model.fit(X_train, y_train);


In [14]:
# feature_words = count_vect.get_feature_names()
# n = 20 #number of top words associated with the category that we wish to see
# target_names[categories] = 1
# for cat in range(len(categories)): #categories == ['negative', 'positive']
#     # print(f"\nTarget: {cat}")
#     # print(f"\nname: ", target_names[cat])
#     print(f"\nTarget: {cat}, name:", target_names[cat])
#     log_prob = nb_model.feature_log_prob_[cat]
#     i_topn = np.argsort(log_prob)[::-1][:n]
#     features_topn = [feature_words[i] for i in i_topn]
#     print(f"Top {n} tokens: ", features_topn)


In [15]:
y_predicted = nb_model.predict(X_test)
# X_test[0]

In [16]:
# y_predicted

In [17]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predicted)


0.3738656987295826

In [18]:
y_pred_proba = nb_model.predict_proba(X_test)[:,1]
y_pred_proba

array([0.03595463, 0.63162074, 0.00570894, ..., 0.00992121, 0.06584134,
       0.01759775])

In [24]:
thresh=0.7
y_pred = (y_pred_proba>=thresh).astype(int)
# y_pred

In [25]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

0.7783251231527093

In [26]:
# pickle and save the model 
import pickle
save_documents = open("pickled_algos/pickled_nb_tweets.pickle","wb")
pickle.dump(nb_model, save_documents)
save_documents.close()

tfidf = tfidf_transformer
save_documents = open("pickled_algos/tfidf_transformer_tweets.pickle", "wb")
pickle.dump(tfidf, save_documents)
save_documents.close()

count_vect = count_vect
save_documents = open("pickled_algos/count_vect_tweets.pickle", "wb")
pickle.dump(count_vect, save_documents)
save_documents.close()



In [32]:
def predict_one_twitter(input):
    with open('pickled_algos/pickled_nb.pickle', 'rb') as f:
        model = pickle.load(f)

    with open('pickled_algos/tfidf_transformer.pickle', 'rb') as f:
        tfidf = pickle.load(f)

    with open('pickled_algos/count_vect.pickle', 'rb') as f:
        cv = pickle.load(f)

  
    cv_transformed = cv.transform(string_pred) #counts how many words
    tfidf_transformed = tfidf.transform(cv_transformed)  #tf == cv . 
    string_predicted = model.predict(tfidf_transformed) 
    res = str(string_predicted[0])
    if res == '0':
        res = ('Negative Sentiment')
    else:
        res = ("Positive Sentiment")
    return res

# string_pred = ['this girl is a foster pit and has none of her teeth']
string_pred = ['fill out an online form to find this male puppy a forever home']

predict_one_twitter(string_pred)

'Positive Sentiment'